In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import ( ModelSpec as MS, summarize )

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Obtenção de dados: ler algum dataframe do Pandas. A forma mais fácil é de fato usar algum dataset do ISLP.

In [ ]:
hitters = load_data('Hitters')
hitters.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


Operações em Dataframe: Usar operador [], funções loc, iloc.
Obter índices por objeto .index

In [ ]:
hitters['AtBat'] # Retorna uma series
hitters[['Hits','RBI']] # Retorna um subdataframe
hitters.iloc[1:5,3:8] # Retorna um subdataframe
x = hitters.iloc[1,3:8]; hitters.iloc[1:5,6] # Retorna uma series
hitters.iloc[1,3] # Retorna um valor do mesmo tipo da Series coluna
hitters.index # Retorna os índices das linhas (ex: linha i -> i)
hitters.columns # Retorna os índices das colunas
hitters.loc['RBI':'Walks'] # Retorna um subdataframe
hitters.loc[~(hitters.HmRun < 7) ^ (hitters.Years > 14), ['RBI','Walks','CRuns']] # Retorna um subdataframe

hitters = hitters[~np.isnan(hitters.Salary)]

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


Construir X e Y. Podemos usar modelspec do ISLP ou fazer manualmente.

In [138]:
X_train, X_test, Y_train, Y_test = train_test_split(hitters.drop(columns=['Salary']),hitters['Salary'],test_size=0.2,random_state=1)

# ISLP
design = MS(X_train.columns)
X = design.fit_transform(X_train)
Y = Y_train

# Manual
from statsmodels.tools import add_constant
X2 = pd.get_dummies(add_constant(X_train),drop_first = True)
Y2 = Y_train

Treino do modelo. Podemos usar statsmodels ou scikit-learn.

In [ ]:
# Statsmodels
model = sm.OLS(Y, X)
res = model.fit()

# Scikit
model2 = LinearRegression()
res2 = model2.fit(X,Y)

print(summarize(res),end="\n\n\n")
print(res2.intercept_,res2.coef_[1:])

                  coef  std err      t  P>|t|
intercept     156.6631  100.855  1.553  0.122
AtBat          -1.8766    0.750 -2.502  0.013
Hits            6.5895    2.673  2.465  0.015
HmRun           0.2546    7.236  0.035  0.972
Runs           -0.4457    3.356 -0.133  0.894
RBI             0.0389    2.982  0.013  0.990
Walks           4.5628    2.034  2.244  0.026
Years           6.1463   13.450  0.457  0.648
CAtBat         -0.3564    0.155 -2.296  0.023
CHits           0.9897    0.735  1.347  0.180
CHmRun          1.5903    1.765  0.901  0.369
CRuns           0.7658    0.831  0.921  0.358
CRBI            0.2674    0.735  0.364  0.716
CWalks         -0.4454    0.360 -1.238  0.217
League[N]      51.9573   94.174  0.552  0.582
Division[W]  -116.5753   44.527 -2.618  0.010
PutOuts         0.2871    0.086  3.326  0.001
Assists         0.1426    0.247  0.576  0.565
Errors          1.6549    4.783  0.346  0.730
NewLeague[N]   -3.2591   93.936 -0.035  0.972


156.66310964409598 [-1.87656585e

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.565
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     13.00
Date:                Fri, 19 Sep 2025   Prob (F-statistic):           3.01e-25
Time:                        10:04:18   Log-Likelihood:                -1490.1
No. Observations:                 210   AIC:                             3020.
Df Residuals:                     190   BIC:                             3087.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
intercept      156.6631    100.855      1.553      0.122     -42.276     355.602
AtBat           -1.8766      0.750     -2.502      0.013      -3.356      -0.397
Hits             6.5895      2.673      2.465      0.015       1.316      11.863
HmRun            0.2546      7.236      0.035      0.972     -14.019      14.528
Runs            -0.4457      3.356     -0.133      0.894      -7.065       6.173
RBI              0.0389      2.982      0.013      0.990      -5.843       5.921
Walks            4.5628      2.034      2.244      0.026       0.551       8.574
Years            6.1463     13.450      0.457      0.648     -20.383      32.676
CAtBat          -0.3564      0.155     -2.296      0.023      -0.663      -0.050
CHits            0.9897      0.735      1.347      0.180      -0.460       2.439
CHmRun           1.5903      1.765      0.901      0.369      -1.891       5.072
CRuns            0.7658      0.831      0.921      0.358      -0.874       2.406
CRBI             0.2674      0.735      0.364      0.716      -1.182       1.717
CWalks          -0.4454      0.360     -1.238      0.217      -1.155       0.264
League[N]       51.9573     94.174      0.552      0.582    -133.803     237.717
Division[W]   -116.5753     44.527     -2.618      0.010    -204.406     -28.745
PutOuts          0.2871      0.086      3.326      0.001       0.117       0.457
Assists          0.1426      0.247      0.576      0.565      -0.345       0.631
Errors           1.6549      4.783      0.346      0.730      -7.780      11.090
NewLeague[N]    -3.2591     93.936     -0.035      0.972    -188.550     182.032
==============================================================================
Omnibus:                       80.814   Durbin-Watson:                   1.728
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              565.389
Skew:                           1.284   Prob(JB):                    1.69e-123
Kurtosis:                      10.617   Cond. No.                     2.32e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""